In [1]:
import pandas as pd
pd.set_option('display.max_colwidth',100)


In [2]:
df = pd.read_csv('models.csv')

In [3]:
df.head()

,,Model Name,Provider,Release Date,Parameters,Architecture,Training Data Cutoff,Key Features
GPT-4,OpenAI,2023-03,>1T,Transformer,2023-01,Multimodal (text + image),better reasoning,long context
GPT-3.5,OpenAI,2022-11,175B,Transformer,2021-09,Faster and cheaper than GPT-3,improved chat,NaN
Claude 2,Anthropic,2023-07,Unknown,Constitutional AI,2023-01,Long context window (100k tokens),safety-focused,NaN
Claude Instant,Anthropic,2023-03,Unknown,Constitutional AI,2023-01,Faster,cheaper Claude variant,NaN
LLaMA 2,Meta,2023-07,7B / 13B / 70B,Transformer,2023-06,Open-source,strong fine-tuning ability,NaN


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 11 entries, ('GPT-4', 'OpenAI') to ('Falcon 180B', 'Technology Innovation Institute')
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Model Name            11 non-null     object
 1   Provider              11 non-null     object
 2   Release Date          11 non-null     object
 3   Parameters            11 non-null     object
 4   Architecture          11 non-null     object
 5   Training Data Cutoff  10 non-null     object
 6   Key Features          2 non-null      object
dtypes: object(7)
memory usage: 1.4+ KB


In [5]:
from sentence_transformers import SentenceTransformer

d:\Langchain_project\venv.rockyfinance\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
encoder = SentenceTransformer('all-mpnet-base-v2')

In [7]:
vectors = encoder.encode(df.Architecture)

d:\Langchain_project\venv.rockyfinance\Lib\site-packages\sentence_transformers\SentenceTransformer.py:1015: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sentences_sorted = [sentences[int(idx)] for idx in length_sorted_idx]


In [8]:
print(vectors.shape)
print(df.shape)

(11, 768)
(11, 7)


In [9]:
dim = vectors.shape[1]
dim

768

In [10]:
# creating an empty index using faiss
import faiss

index = faiss.IndexFlatL2(dim)

index.add(vectors)

In [11]:
search_query = "which models are better tahn GPT?"
vec = encoder.encode(search_query)
vec.shape

(768,)

In [12]:
#since index.search expects a 2 dim array we will convert the vec to 2d using numpy library
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [13]:
distances , I =index.search(svec, k =3)

In [14]:
print(I)
print(distances)

[[1 4 9]]
[[0.7205563 1.5879384 1.5980325]]


In [15]:
df.iloc[I[0]]

,,Model Name,Provider,Release Date,Parameters,Architecture,Training Data Cutoff,Key Features
GPT-3.5,OpenAI,2022-11,175B,Transformer,2021-09,Faster and cheaper than GPT-3,improved chat,NaN
LLaMA 2,Meta,2023-07,7B / 13B / 70B,Transformer,2023-06,Open-source,strong fine-tuning ability,NaN
Command R+,Cohere,2024-04,Unknown,Transformer,2024-02,RAG-optimized,strong retrieval abilities,NaN


In [16]:
search_query

'which models are better tahn GPT?'